In [ ]:
import pandas as pd

In [ ]:
data="""Rank Name Age Height
(cm)
Weight
(kg) Career Titles
1 Novak Djokovic 35 188 77 93
2 Carlos Alcaraz 19 183 74 7
3 Stefanos Tsitsipas 24 193 90 9
4 Casper Ruud 24 183 77 9
5 Andrey Rublev 25 188 75 12
6 Rafael Nadal 36 185 85 92
7 Taylor Fritz 25 196 86 5
8 Daniil Medvedev 27 198 83 16
9 Felix Auger-Aliassime 22 193 88 4
10 Holger Rune 19 188 77 3"""
data = data.splitlines()
data = [" ".join(data[:4])] + data[4:]
columns = data[0].split()
columns = columns[0:3] + [" ".join(columns[3:5])] +[" ".join(columns[5:7])] + [" ".join(columns[7:])]
data = [i.split() for i in data[1:]]
data = [x[:1] + [" ".join(x[1:3])] + x[3:] for x in data]
# display(data, columns)
data = pd.DataFrame(data, columns=columns)
for col,dtype in zip(columns, [int, str, int, int, int, int]):
    data[col] = data[col].astype(dtype)

display(data.rename({col: f"{col} ({data[col].dtype})" for col in data.columns}, axis=1))

In [ ]:
## (a)
# Low level implementation for better understanding
n_bins = 2
size_bins = len(data) // n_bins

# Take note that this looks like it creates a list of n_bins empty lists, 
# but it actually creates a list of n_bins references to the same empty list.
# bins =[[]]*n_bins
# Therefore this
bins = [[] for _ in range(n_bins)]
last_age, last_bin = 0, 0
for i, row in enumerate(data.sort_values("Age").itertuples()):
    if row.Age == last_age:
        bins[last_bin].append(row)
    else:
        bins[i//size_bins].append(row)
        last_age, last_bin = row.Age, i//size_bins

bin_bounds = [(bin[0].Age, bin[-1].Age) for bin in bins]
for bin, (lower, upper) in zip(bins, bin_bounds):
    print(f"Age range: {lower}-{upper}")
    for row in bin:
        print(f"{row.Rank} {row.Name} {row.Age}")
    print()

In [ ]:
# Equal frequency binning making more use of libraries, feel free to experiment with the parameters
from sklearn.preprocessing import KBinsDiscretizer
discretizer = KBinsDiscretizer(n_bins=2, encode='ordinal', strategy='quantile')

cell_df = data.copy()
cell_df['Age Bin'] = discretizer.fit_transform(cell_df[['Age']]).astype(int)

for bin in sorted(cell_df['Age Bin'].unique()):
    bin_df = cell_df[cell_df['Age Bin'] == bin]
    print(f"Bin {int(bin)+1}: [{bin_df['Age'].min()}, {bin_df['Age'].max()}] len={len(bin_df)}")
    display(bin_df.sort_values('Age'))

In [ ]:
## (b)
# Min-Max scaling low level implementation

min_scale, max_scale = 1, 10
min_raw, max_raw = 3, 93

# min_raw       (max_raw-min_raw)     max_raw
#    |-----------------------------------|
# min_scale   (max_scale-min_scale)   max_scale

raw_to_scaled = lambda x: (x-min_raw)*(max_scale-min_scale)/(max_raw-min_raw) + min_scale
scaled_to_raw = lambda x: (x-min_scale)*(max_raw-min_raw)/(max_scale-min_scale) + min_raw

assignment = [93,7,9]
for x in assignment:
    print(f"{x} -> {raw_to_scaled(x)}")

In [ ]:
# Min-Max scaling using libraries
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler(feature_range=(1, 10))
cell_df = data.copy()
col = 'Career Titles'
cell_df.insert(cell_df.columns.get_loc(col)+1, f"{col} Scaled", scaler.fit_transform(cell_df[[col]]))

if False:
    display(cell_df[cell_df["Rank"].apply(lambda x: x in [1,2,3])])
else:
    display(cell_df)

In [ ]:
## (c) Outliers
from sklearn.preprocessing import StandardScaler
import re
df_c = data.copy()
scaler_c = StandardScaler()
numeric_columns = ['Age', 'Height (cm)', 'Weight (kg)', 'Career Titles']
scaler_c.fit(df_c[numeric_columns]);
df_c[numeric_columns] = scaler_c.transform(df_c[numeric_columns])
for col in df_c.columns:
    df_c[col] = df_c[col].round(2)
    name = col
    if match:=re.search(r"\s*\(.*\)\s*$", name):
        name = name[:match.start()]
    name = name.replace(" ", "_").lower()
    df_c.rename({col: name}, axis=1, inplace=True)
display(df_c)


In [ ]:
# Low level implementation
abs_error_threshold = 0.3
prediction_f = lambda x: (0.92*x.age) - (0.24 * x.height) - (0.02 * x.weight)
id_by_rank = [1,2,3]
cell_data = [row for row in df_c.itertuples() if row.rank in id_by_rank]
for row in cell_data:
    prediction = prediction_f(row)
    error = abs(row.career_titles - prediction)
    print(f"Rank {row.rank}:")
    print(f"\t{row.name} has a predicted career titles of {prediction:.2f} vs actual {row.career_titles:.2f}.")
    print(f"\tError: {error:.2f} {'(outlier)' if error > abs_error_threshold else ''}")


In [ ]:
# High level implementation
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import numpy as np
import seaborn as sns

cell_df = df_c.copy()
abs_error_threshold = 0.3
model = LinearRegression()
input_cells = ['age', 'height', 'weight']
target_cell = 'career_titles'
X = cell_df[input_cells]
y = cell_df[target_cell]
model.fit(X, y)
predictions = model.predict(X)
errors = np.abs(predictions - y)
cell_df[f'predicted_{target_cell}'] = predictions
cell_df[f'predicted_{target_cell}_reversed'] = (
    scaler_c.inverse_transform(cell_df[input_cells + [f'predicted_{target_cell}']]).T[0].T.round(0)
)
cell_df[f'predicted_{target_cell}_reversed'] = cell_df[f'predicted_{target_cell}_reversed'].astype(int)
cell_df['error'] = errors
cell_df['outlier'] = errors > abs_error_threshold

out_str = [f"{target_cell} ="]
for var,factor in zip(input_cells,model.coef_):
    if factor > 0:
        if len(out_str) > 2:
            out_str.append("+")
    else:
        out_str.append("-")
        factor = -factor
    out_str.append(f"{factor:.2f}*{var}")
out_str.append(f" + {model.intercept_:.2f}")
print(" ".join(out_str))
display(cell_df.round(2))

cell_df['rank'] = cell_df['rank'].astype(str)
sns.scatterplot(data=cell_df, x='rank', y=target_cell, color='blue', label='Actual')
sns.scatterplot(data=cell_df, x='rank', y=f'predicted_{target_cell}', color='red', label='Predicted')